In [1]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver

from bs4 import BeautifulSoup 
import requests 

from tqdm import tqdm
import time
import random
from copy import deepcopy

import pickle

In [2]:
import pandas as pd
import numpy as np

In [ ]:
pip install tqdm

# Scrape

## Get Attractions

In [ ]:
attraction_links = []

In [ ]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [ ]:
miss_links = []

In [ ]:
soup = BeautifulSoup(driver.page_source)

### Finding Links Version 1

In [ ]:
attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})

In [ ]:
for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])

### Finding Links Version 2

In [ ]:
soup = BeautifulSoup(driver.page_source)

In [ ]:
attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))

In [ ]:
# Will only work if there's the second type of link in the webpage

In [ ]:
for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))

In [ ]:
len(attraction_links)

### Finding Next Button

In [ ]:
for i in soup.find_all('a', href=True):
    if i.text == '\nNext\n':
        link = 'https://www.tripadvisor.com/'+i['href']
        driver.get(link)
        break

In [ ]:
for i in soup.find_all('a', href=True):
    if i.text == 'Next':
        link = 'https://www.tripadvisor.com/'+i['href']
        driver.get(link)
        break

### Code to run

In [ ]:
attraction_links = []
miss_links = []

##

driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

##

# Don't run attraction_links in same location
# attraction_links = []

#319 things to do in Athens
#Range= number of pages to go to
for i in tqdm(range(12)):
#if there is a next button, continue to click and scrape
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        print()
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
              
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    sequence = [x/10 for x in range(5, 10)]
    time.sleep(random.choice(sequence))

##

for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)
        
##

fixed_links = []

##

for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

##

df = pd.DataFrame(fixed_links)

##

df.to_pickle('athens_links.pkl')

##




In [ ]:
attraction_links = []
miss_links = []

In [ ]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [ ]:
# Don't run attraction_links in same location
# attraction_links = []

#319 things to do in Athens
#Range= number of pages to go to
for i in tqdm(range(12)):
#if there is a next button, continue to click and scrape
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        print()
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
              
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    sequence = [x/10 for x in range(5, 10)]
    time.sleep(random.choice(sequence))

### Fix Attraction Links

In [ ]:
attraction_links = list(set(attraction_links))

In [ ]:
len(attraction_links)

In [ ]:
#creating a test copy
t = deepcopy(attraction_links)

In [ ]:
for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)

In [ ]:
len(attraction_links)

In [ ]:
fixed_links = []

In [ ]:
for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

In [ ]:
fixed_links

In [ ]:
df = pd.DataFrame(fixed_links)

In [ ]:
df.to_pickle('athens_links.pkl')

In [ ]:
#convert dataframe to list, remember to call on column(0)
athens_links = df[0].tolist()

In [3]:
df = pd.read_pickle('athens_links.pkl')

In [4]:
athens_links = df[0].tolist()

# Get Info From Attractions

In [31]:
athens_links[0]

'https://www.tripadvisor.com/Attraction_Review-g189400-d12642863-Reviews-Gate_of_Medrese-Athens_Attica.html'

In [ ]:
driver.get(athens_links[0])

In [34]:
driver.find_element_by_xpath('//*[@id="HEADING"]').text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="HEADING"]"}
  (Session info: chrome=79.0.3945.130)


## Definitions

In [313]:
def find_name(driver):
    return driver.find_element_by_xpath('//*[@id="HEADING"]').text

def find_no_things_to_do(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[1]/div/div[2]/div/span').text

#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[1]/div/span').text

# Find Address
def find_address(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[2]/div/div[1]/div/div').text


#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/span').text

#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/a[2]').text

#Find About
def about_box(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_overview_resp_ar_responsive_0"]/div/div/div[2]').text

#Find The Popular Mentions in comments
def popular_mentions(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_detail_filters_ar_responsive_0"]/div/div[2]/div[3]/div').text.split('\n')

#Find 10 Most recent comments 
def comments(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_reviews_list_resp_ar_responsive_0"]/div').text

def reviews(driver):
    return 

In [315]:
comments(driver)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="taplc_location_reviews_list_resp_ar_responsive_0"]/div"}
  (Session info: chrome=79.0.3945.130)


In [314]:
number_of_reviews(driver)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/a[2]"}
  (Session info: chrome=79.0.3945.130)


## Secondary Definitions

In [171]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[1]/div/div/div[1]/h1').text

'Greek Folk Art Museum (Museo Ellinikis Laikis Technis)'

In [173]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[1]/div/div/span').text

'#131 of 319 things to do in Athens'

In [177]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[3]/span/div').text

'Museums, Art Museums'

In [179]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[2]/div/div[2]').text

'"House on Panos Street - Man and Tools"\nThis is a fascinating little museum and only 3 Euros. It is on the street between the Acropolis and...Read more\nReviewed June 26, 2018\n817kathye\n"Nothing here."\nMusuem empty but still charged admission fee. Only thing of interest was the old bath house but not ...Read more\nReviewed April 2, 2017\npartial_eclipse'

In [181]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[1]/div/div/div[1]/div/span[2]').text

'42 Reviews'

In [182]:
driver.find_element_by_xpath('//*[@id="component_22"]/div[3]/div').text

"Traveller rating\nExcellent 16\nVery Good 18\nAverage 5\nPoor 0\nTerrible 3\nTraveller type\nFamilies\nCouples\nSolo\nBusiness\nFriends\nTime of year\nMar-May\nJun-Aug\nSep-Nov\nDec-Feb\nLanguage\nAll languages\nEnglish(28)\nGreek(3)\nFrench(2)\nMore\nKathy E wrote a review Jun. 2018\nPortland, Oregon2,696 contributions148 helpful votes\nHouse on Panos Street - Man and Tools\nThis is a fascinating little museum and only 3 Euros. It is on the street between the Acropolis and the Ancient Agora. We were in need of a few minutes sitting down (and a clean restroom) and went into this little gem. It won't be a highlight of your trip, but it is…\nRead more\nDate of experience: May 2018\nHelpful\nShare\npartial_eclipse wrote a review Apr. 2017\n32 contributions7 helpful votes\nNothing here.\nMusuem empty but still charged admission fee. Only thing of interest was the old bath house but not worth paying to get in.\nRead more\nDate of experience: April 2017\nHelpful\nShare\nVanessa_Fou wrote a 

In [ ]:
driver.find_element_by_xpath('').text

In [ ]:
driver.find_element_by_xpath('').text

In [ ]:
driver.find_element_by_xpath('').text

In [ ]:
driver.find_element_by_xpath('').text

In [334]:
def find_name(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/h1').text

def find_no_things_to_do(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[1]/div/div/span').text

#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[3]/span/div').text

# Find Address
def find_address(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[2]/div/div[1]/div/div').text


#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[2]/div/div[2]').text

#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/div/span[2]').text
#Find About
def about_box(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_overview_resp_ar_responsive_0"]/div/div/div[2]').text

#Find The Popular Mentions in comments
def popular_mentions(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_detail_filters_ar_responsive_0"]/div/div[2]/div[3]/div').text.split('\n')

#Find 10 Most recent comments 
def comments(driver):
    return driver.find_element_by_xpath('//*[@id="component_19"]/div[3]/div').text

#click the next button

def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break
                
                
def morebutton(driver):
    try:
        more_button = driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
        return more_button.click()
    except:
        return False
    
# def morebutton(driver):
#     try: 
#         more_button = driver.find_element_by_class_name('ulBlueLinks')
#         return more_button.click()
#     except:
#         return False
    
    
            

# def reviews(driver):
#     return driver.find_element_by_css_selector('#component_21 > div > div > div.attractions-attraction-review-atf-AttractionReviewATFLayout__atf_left_column--1ogzd > div.attractions-attraction-review-atf-AttractionReviewATFLayout__atf_component--2Qflo.attractions-attraction-review-atf-AttractionReviewATFLayout__header_container--dAJtk > div > div > div.ui_column.is-12-tablet.is-10-mobile.attractions-attraction-review-header-attraction-review-header__order0--1bYeG.attractions-attraction-review-header-attraction-review-header__columns--2j94i > h1').text

In [335]:

driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/div/span[2]').text


'280 Reviews'

In [336]:
number_of_reviews(driver)

'280 Reviews'

In [337]:
find_summary_overview(driver)

"Every beer lover who visits Santorini must do a tasting at one of the best breweries I've ever experienced!"

In [339]:
comments(driver)

"Traveller rating\nExcellent 155\nVery Good 80\nAverage 26\nPoor 14\nTerrible 5\nTraveller type\nFamilies\nCouples\nSolo\nBusiness\nFriends\nTime of year\nMar-May\nJun-Aug\nSep-Nov\nDec-Feb\nLanguage\nAll languages\nEnglish(218)\nItalian(20)\nGerman(7)\nMore\nPopular mentions\nAll reviewscrazy donkeyyellow donkeyt shirtsfree tastingbeer tastingdonkey beerslove beermicro brewerynice beergreat breweryquick visitsmall batchexo goniaworth a stopbuy a bottlebus stoptasting areaour honeymoonmerchandiseipa\nAnn Shen wrote a review Nov. 2019\n17 contributions1 helpful vote\nSmall batch beers\nThe beers made at Santorini brewing company are quite interesting and unique. Our pours were generous and we learnt a lot about each of their beers, the best of which was definitey Slow Donkey. Worth a stop over and a beer palate cleanser in between all of the wine.\nRead more\nDate of experience: October 2019\nHelpful\nShare\nLLV280Z wrote a review Oct. 2019\nDaytona Beach, Florida75 contributions10 help

In [324]:
def morebutton(driver):
    try:
        more_button = driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
        return more_button.click()
    except:
        return False
    

In [210]:
def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break

In [326]:
comments(driver)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="component_22"]/div[3]/div"}
  (Session info: chrome=79.0.3945.130)


In [322]:
driver.find_element_by_xpath('//*[@id="component_19"]/div[3]/div/div[3]').text

"Manuela wrote a review Jan. 2020\nTurin, Italy13 contributions1 helpful vote\nFantastic articles and very professional staff\nI discovered Mati during a holiday in Santorini some years ago. Since from the beginning I thought that it was fantastic. Few weeks ago I've bought a Christmas present contacting Mati by email. Mrs Diana was very kind and very professional, she gave me excellent suggestions .…\nRead more\nDate of experience: January 2020\nHelpful\nShare\nResponse from Alex K, General Manager at MATI Art Gallery\nResponded 2 weeks ago\nGood afternoon, Thank you very much for kind words, we really appreciate your feedback! Thank you again for the purchase and of course if you need anything we are here to help! All the best, Alex Kypris\nRead more"

In [319]:
driver.find_element_by_xpath('//*[@id="component_22"]/div[3]/div')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="component_22"]/div[3]/div"}
  (Session info: chrome=79.0.3945.130)


In [209]:
next_button.click()

In [203]:
driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g').click()

## Third Set of Definitions (by class)

In [ ]:
def find_name(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/h1').text

def find_no_things_to_do(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[1]/div/div/span').text

#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[3]/span/div').text

# Find Address
def find_address(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[2]/div/div[1]/div/div').text


#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[2]/div/div[2]').text

#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/div/span[2]').text
#Find About
def about_box(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_overview_resp_ar_responsive_0"]/div/div/div[2]').text

#Find The Popular Mentions in comments
def popular_mentions(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_detail_filters_ar_responsive_0"]/div/div[2]/div[3]/div').text.split('\n')

#Find 10 Most recent comments 
def comments(driver):
    return driver.find_element_by_xpath('//*[@id="component_19"]/div[3]/div').text

#click the next button

def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break
                
                
def morebutton(driver):
    try:
        more_button = driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
        return more_button.click()
    except:
        return False
    
# def morebutton(driver):
#     try: 
#         more_button = driver.find_element_by_class_name('ulBlueLinks')
#         return more_button.click()
#     except:
#         return False
    
    
            

# def reviews(driver):
#     return driver.find_element_by_css_selector('#component_21 > div > div > div.attractions-attraction-review-atf-AttractionReviewATFLayout__atf_left_column--1ogzd > div.attractions-attraction-review-atf-AttractionReviewATFLayout__atf_component--2Qflo.attractions-attraction-review-atf-AttractionReviewATFLayout__header_container--dAJtk > div > div > div.ui_column.is-12-tablet.is-10-mobile.attractions-attraction-review-header-attraction-review-header__order0--1bYeG.attractions-attraction-review-header-attraction-review-header__columns--2j94i > h1').text

In [404]:
def find_name(driver):
    return driver.find_element_by_class_name('ui_header').text

def find_no_things_to_do(driver):
    return driver.find_element_by_class_name('attractions-attraction-review-header-attraction-review-header__popIndex--H1_nL').text

#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_class_name('attractions-attraction-review-header-AttractionLinks__detail--2-xvX').text

# Find Address
def find_address(driver):
    return driver.find_element_by_class_name('attractions-contact-card-ContactCard__contactRow--3Ih6v').text

def find_meta_data(driver):
    return driver.find_element_by_class_name('masthead_h1').text

#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_class_name('attractions-review-highlights-ReviewHighlights__snippetBlock--3KvE9').text

#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_class_name('attractions-attraction-review-header-attraction-review-header__ratingContainer--1lMqm').text

#Find About
def about_box(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_overview_resp_ar_responsive_0"]/div/div/div[2]').text

#Find The Popular Mentions in comments
def popular_mentions(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_detail_filters_ar_responsive_0"]/div/div[2]/div[3]/div').text.split('\n')

#Find 10 Most recent comments 
def comments(driver):
    return driver.find_element_by_class_name('ppr_priv_community_content').text

def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break
                
def morebutton(driver):
    try:
        more_button = driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
        return more_button.click()
    except:
        return False

### Testing Selenium

In [ ]:
attractions= []

In [ ]:
attraction_features = {}
# add to dictionary
try:
    attraction_features['name'] = find_name(driver)
except:
    attraction_features['name'] = None

# add to dictionary
try:
    attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
except:
    attraction_features['no_things_to_do'] = None

# add to dictionary
try:
    attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
except:
    attraction_features['sights_landmarks'] = None

# add to dictionary
try:
    attraction_features['address'] = find_address(driver)
except:
    attraction_features['address'] = None

# add to dictionary
try:
    attraction_features['summary_overview'] = find_summary_overview(driver)
except:
    attraction_features['summary_overview'] = None

# add to dictionary
try:
    attraction_features['num_reviews'] = number_of_reviews(driver)
except:
    attraction_features['num_reviews'] = None

# add to dictionary
try:
    attraction_features['about'] = about_box(driver)
except:
    attraction_features['about'] = None

# add to dictionary
try:
    attraction_features['popular_mentions'] = popular_mentions(driver)
except:
    attraction_features['popular_mentions'] = None

    
try:
    more_button = driver.find_element_by_class_name('ulBlueLinks')
    more_button.click()
    
except:
    pass
    
# add comments to dictionary
driver.implicitly_wait(5)
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

    
    
try:
    for i in driver.find_elements_by_class_name('next'):
        if i.text=='Next':
            next_button = i
            next_button.click()
            break
    
#add to list of dictionaries
attractions.append(attraction_features)

sequence = [x/10 for x in range(10, 15)]
time.sleep(random.choice(sequence))

## Click More on Comments

In [ ]:
comments(driver)

In [ ]:
attractions

## Comments 

In [ ]:
more_button = driver.find_element_by_class_name('ulBlueLinks')

In [ ]:
more_button.click()

In [ ]:
driver.implicitly_wait(10)

## Click Next Comments

In [8]:
for i in driver.find_elements_by_class_name('next'):
    if i.text=='Next':
        next_button = i
        next_button.click()
        break

NameError: name 'driver' is not defined

## For Loop Comments

In [ ]:
attraction_features['comments']= comments(driver)

In [ ]:
attraction_features['comments']+=(comments(driver))

In [ ]:
attraction_features['comments']+=''

In [ ]:
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None
    
    
    
    
# last_comment == comments(driver)

last_comment = comments(driver)

for i in range(3):    
    #click next
    if driver.find_element_by_class_name('next') == 'Next':
        next_button = driver.find_element_by_class_name('next')
        next_button.click()
    else:
        break

    #click more
    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))
    except:
        pass

    #check if this is the same as last time
    if last_comment == comments(driver):
        break
    else:
        try:
            attraction_features['comments']+=(comments(driver))
        except:
            attraction_features['comments']+=''

        last_comment = comments(driver)

In [ ]:
# first click more
# then get comments!


sequence = [x/10 for x in range(15, 30)]

#loop through 1st 3 pages if they have a next
for next_comments in range(4):
    if driver.find_element_by_class_name('next')== None:
        break
    else:
        for i in driver.find_elements_by_class_name('next'):
            if i.text=='Next':
                next_button = i
                next_button.click()

    time.sleep(random.choice(sequence))

    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))

    except:
        pass


    #get comments
    # add to dictionary

    driver.implicitly_wait(5)
    try:
        attraction_features['comments']+=(comments(driver))
    except:
        attraction_features['comments']+=''

    time.sleep(random.choice(sequence))

## Selenium Code To Scrape

In [139]:
sequence = [x/10 for x in range(20, 35)]

In [ ]:
attractions = []
test = athens_links[5:]
len(athens_links)
len(test)

In [238]:
attractions = []

In [258]:
test = athens_links[5:]

In [259]:
len(test)

392

In [255]:
len(athens_links)

397

In [201]:
driver.get(test[0])

In [244]:
number_of_reviews(driver)

'1 Review'

In [166]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
# driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [197]:
morebutton(driver)

False

In [260]:
for i in tqdm(test):
    driver.get(i)
    time.sleep(random.choice(sequence))
    time.sleep(random.choice(sequence))

    
    attraction_features = {}
    
    attraction_features['link'] = i
    
    # add to dictionary
    try:
        attraction_features['name'] = find_name(driver)
    except:
        attraction_features['name'] = None

    # add to dictionary
    try:
        attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
    except:
        attraction_features['no_things_to_do'] = None

    # add to dictionary
    try:
        attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
    except:
        attraction_features['sights_landmarks'] = None

    # add to dictionary
    try:
        attraction_features['address'] = find_address(driver)
    except:
        attraction_features['address'] = None

    # add to dictionary
    try:
        attraction_features['summary_overview'] = find_summary_overview(driver)
    except:
        attraction_features['summary_overview'] = None

    # add to dictionary
    try:
        attraction_features['num_reviews'] = number_of_reviews(driver)
    except:
        attraction_features['num_reviews'] = None

    # add to dictionary
    try:
        attraction_features['about'] = about_box(driver)
    except:
        attraction_features['about'] = None

    # add to dictionary
    try:
        attraction_features['popular_mentions'] = popular_mentions(driver)
    except:
        attraction_features['popular_mentions'] = None
    
    
    try:
        morebutton(driver)
        time.sleep(random.choice(sequence)/2)
    except:
        pass
    
    time.sleep(random.choice(sequence))
    
    # add comments to dictionary
    
    try:
        attraction_features['comments'] = comments(driver)
    except:
        attraction_features['comments'] = None
    
    
    # last_comment == comments(driver)
    try:
        first_comment = comments(driver)
    except:
        pass
    
    

    
    #load more comments if there is any
    for i in (range(3)):

        try:
            if nextbutton(driver) == False:
                break
            else:
                nextbutton(driver)
                time.sleep(random.choice(sequence))

                try:
                    morebutton(driver)
                except:
                    pass

                time.sleep(random.choice(sequence))

                try:
                    if first_comment == comments(driver):
                        break
                    else:
                        first_comment = comments(driver)
                        
                except:
                    pass
                
                try:
                    attraction_features['comments']+=(comments(driver))
                except:
                    attraction_features['comments']+=''    
                    
                
        except:
            break
        
        
        try:
            #checks to make sure we're not recopying comments
            if last_comment == comments(driver):
                break
        except:
            pass
    #add to list of dictionaries
    attractions.append(attraction_features)






  0%|          | 0/392 [00:00<?, ?it/s]


  0%|          | 1/392 [00:15<1:41:24, 15.56s/it]


  1%|          | 2/392 [00:31<1:42:31, 15.77s/it]


  1%|          | 3/392 [00:51<1:50:17, 17.01s/it]


  1%|          | 4/392 [01:09<1:51:23, 17.23s/it]


  1%|▏         | 5/392 [01:39<2:16:26, 21.15s/it]


  2%|▏         | 6/392 [02:06<2:27:34, 22.94s/it]


  2%|▏         | 7/392 [02:28<2:24:29, 22.52s/it]


  2%|▏         | 8/392 [02:59<2:41:18, 25.21s/it]


  2%|▏         | 9/392 [03:15<2:22:40, 22.35s/it]


  3%|▎         | 10/392 [03:33<2:14:37, 21.15s/it]


  3%|▎         | 11/392 [03:49<2:04:21, 19.59s/it]


  3%|▎         | 12/392 [04:09<2:03:14, 19.46s/it]


  3%|▎         | 13/392 [04:27<2:00:15, 19.04s/it]


  4%|▎         | 14/392 [04:44<1:56:20, 18.47s/it]


  4%|▍         | 15/392 [05:02<1:56:11, 18.49s/it]


  4%|▍         | 16/392 [05:22<1:57:23, 18.73s/it]


  4%|▍         | 17/392 [05:53<2:20:13, 22.43s/it]


  5%|▍         | 18/392 [06:12<2:14:22, 21.56s/it]


  5%|▍    

 33%|███▎      | 131/392 [44:42<1:19:38, 18.31s/it]


 34%|███▎      | 132/392 [44:59<1:18:13, 18.05s/it]


 34%|███▍      | 133/392 [45:16<1:16:00, 17.61s/it]


 34%|███▍      | 134/392 [45:32<1:13:24, 17.07s/it]


 34%|███▍      | 135/392 [46:03<1:31:21, 21.33s/it]


 35%|███▍      | 136/392 [46:34<1:42:50, 24.10s/it]


 35%|███▍      | 137/392 [46:49<1:31:48, 21.60s/it]


 35%|███▌      | 138/392 [47:06<1:25:46, 20.26s/it]


 35%|███▌      | 139/392 [47:32<1:31:38, 21.73s/it]


 36%|███▌      | 140/392 [48:03<1:43:11, 24.57s/it]


 36%|███▌      | 141/392 [48:21<1:34:42, 22.64s/it]


 36%|███▌      | 142/392 [48:40<1:29:18, 21.43s/it]


 36%|███▋      | 143/392 [48:57<1:23:41, 20.17s/it]


 37%|███▋      | 144/392 [49:15<1:20:56, 19.58s/it]


 37%|███▋      | 145/392 [49:36<1:22:11, 19.97s/it]


 37%|███▋      | 146/392 [49:57<1:23:39, 20.41s/it]


 38%|███▊      | 147/392 [50:19<1:24:41, 20.74s/it]


 38%|███▊      | 148/392 [50:37<1:21:14, 19.98s/it]


 38%|███▊      | 149/392 [51

 66%|██████▋   | 260/392 [1:27:19<54:07, 24.60s/it]


 67%|██████▋   | 261/392 [1:27:50<57:29, 26.34s/it]


 67%|██████▋   | 262/392 [1:28:20<59:39, 27.53s/it]


 67%|██████▋   | 263/392 [1:28:35<51:23, 23.91s/it]


 67%|██████▋   | 264/392 [1:28:52<46:32, 21.82s/it]


 68%|██████▊   | 265/392 [1:29:13<45:18, 21.41s/it]


 68%|██████▊   | 266/392 [1:29:33<43:56, 20.92s/it]


 68%|██████▊   | 267/392 [1:29:49<40:47, 19.58s/it]


 68%|██████▊   | 268/392 [1:30:05<38:13, 18.49s/it]


 69%|██████▊   | 269/392 [1:30:21<36:04, 17.60s/it]


 69%|██████▉   | 270/392 [1:30:38<35:43, 17.57s/it]


 69%|██████▉   | 271/392 [1:31:10<44:03, 21.85s/it]


 69%|██████▉   | 272/392 [1:31:41<49:04, 24.54s/it]


 70%|██████▉   | 273/392 [1:31:57<43:42, 22.04s/it]


 70%|██████▉   | 274/392 [1:32:15<40:45, 20.72s/it]


 70%|███████   | 275/392 [1:32:32<38:36, 19.80s/it]


 70%|███████   | 276/392 [1:33:04<45:04, 23.32s/it]


 71%|███████   | 277/392 [1:33:20<40:41, 21.23s/it]


 71%|███████   | 278/392 [1:

 99%|█████████▉| 390/392 [2:09:32<00:41, 20.85s/it]


100%|█████████▉| 391/392 [2:09:53<00:20, 20.80s/it]


100%|██████████| 392/392 [2:10:10<00:00, 19.92s/it]


In [261]:
df = pd.DataFrame(attractions)

In [271]:
len(df['comments'][2])

8050

In [274]:
(df['comments'][2])

'Traveller rating\nExcellent 19\nVery Good 1\nAverage 1\nPoor 0\nTerrible 0\nTraveller type\nFamilies\nCouples\nSolo\nBusiness\nFriends\nTime of year\nMar-May\nJun-Aug\nSep-Nov\nDec-Feb\nLanguage\nAll languages\nEnglish(15)\nGreek(3)\nRussian(2)\nJapanese(1)\nPopular mentions\nAll reviewsgalleryartists\nAkag3 wrote a review Oct. 2019\nWestern Australia, Australia26 contributions24 helpful votes\nbeautiful gallery, wonderful people\nWe visited here 3 years ago and bought some earrings and a wedding ring for my husband, which they later posted home for us. Recently my husband lost this ring and we were able to get the item remade. Ersi was so helpful and kept us up to date with the quote and postage. I still…\nRead more\nDate of experience: October 2019\nHelpful\nShare\nResponse from Ersi K, Γενικός Διευθυντής at Eleni Marneri Galerie\nResponded Oct. 9, 2019\nThank you so much for you kind words!! We hope to see you soon!!\nPigadioti wrote a review Mar. 2018\nAthens, Greece103 contributi

,link,name,no_things_to_do,sights_landmarks,address,summary_overview,num_reviews,about,popular_mentions,comments
0,https://www.tripadvisor.com/Attraction_Review-...,Gate of Medrese,#225 of 319 things to do in Athens,"Sights & Landmarks, Points of Interest & Landm...",None,"""FORMER MADRASA ENTRY PORTAL""\nThis doorway is...",6 Reviews,None,None,Traveller rating\nExcellent 1\nVery Good 0\nAv...
1,https://www.tripadvisor.com/Attraction_Review-...,Museum of the History of the Greek Costume of ...,#284 of 319 things to do in Athens,"Museums, History Museums",None,None,5 Reviews,None,None,Traveller rating\nExcellent 0\nVery Good 1\nAv...
2,https://www.tripadvisor.com/Attraction_Review-...,Eleni Marneri Galerie,#100 of 319 things to do in Athens,"Shopping, Museums, Art Galleries",None,None,21 Reviews,None,None,Traveller rating\nExcellent 19\nVery Good 1\nA...
3,https://www.tripadvisor.com/Attraction_Review-...,Stoa Avissinias,#293 of 319 things to do in Athens,"Shopping, Museums, More",None,Stoa Avissinias is a reputable shop in the old...,None,None,None,
4,https://www.tripadvisor.com/Attraction_Review-...,Ikastikos Kyklos Sianti,#220 of 319 things to do in Athens,"Shopping, Museums, Art Galleries",None,"""Ikastikos Kiklos Sianti"" is an art space with...",1 Review,None,None,Traveller rating\nExcellent 1\nVery Good 0\nAv...
...,...,...,...,...,...,...,...,...,...,...
392,https://www.tripadvisor.com/Attraction_Review-...,MOMus - Museum Alex Mylona,#171 of 319 things to do in Athens,"Museums, Art Museums",None,None,5 Reviews,None,None,Traveller rating\nExcellent 3\nVery Good 2\nAv...
393,https://www.tripadvisor.com/Attraction_Review-...,The Jewish Museum of Greece,#18 of 319 things to do in Athens,"Museums, Speciality Museums, History Museums",None,None,171 Reviews,None,None,Traveller rating\nExcellent 104\nVery Good 52\...
394,https://www.tripadvisor.com/Attraction_Review-...,None,None,None,None,None,None,None,None,None
395,https://www.tripadvisor.com/Attraction_Review-...,Penya Barcelonista d'Atenes,#312 of 319 things to do in Athens,"Fun & Games, Sports Complexes",None,None,None,None,None,


In [272]:
df.to_pickle('athens_df.pkl')

In [231]:
df= df.sample(frac=1)

In [236]:
len(df['comments'][25])

2887

In [ ]:
attractions-attraction-review-header-attraction-review-header__reviewCount--3cEMP attractions-attraction-review-header-attraction-review-header__dotted_link--3p26D

In [9]:
from selenium.common.exceptions import NoSuchElementException

In [10]:
#click the next button

def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break
                
def morebutton(driver):
    try: 
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        return more_button.click()
    except:
        return False
            

In [11]:
def morebutton(driver):
    try: 
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        return more_button.click()
    except:
        return False

In [ ]:
morebutton(driver)

In [ ]:
nextbutton(driver)

In [ ]:
try:
    morebutton(driver)
except:
    print('pass')
    pass

In [ ]:
for i in range(3):
    try:
        

In [ ]:
attraction_features

In [ ]:
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

In [ ]:
for i in (range(3)):
    try:
        if nextbutton(driver) != False:
            nextbutton(driver)
            time.sleep(random.choice(sequence))
            
            try:
                morebutton(driver)
            except:
                pass
            
            time.sleep(random.choice(sequence))
            
            try:
                attraction_features['comments']+=(comments(driver))
            except:
                attraction_features['comments']+=''
            
            
            
        else:
            break
        
    except:
        break
    
    
            

#         #click more
#         try:
#             more_button = driver.find_element_by_class_name('ulBlueLinks')
#             more_button.click()
#             time.sleep(random.choice(sequence))
#         except:
#             pass

#         #check if this is the same as last time
#         if last_comment == comments(driver):
# #             print('Last Comment is Current Comment')
#             break
#         else:
#             try:
#                 attraction_features['comments']+=(comments(driver))
#             except:
#                 attraction_features['comments']+=''

#             last_comment = comments(driver)

#             time.sleep(random.choice(sequence))

In [ ]:
    for i in tqdm(range(3)):    
        #click next
        for j in tqdm(driver.find_elements_by_class_name('next')):
            if j.text == 'Next':
                next_button = j
                try:
                    next_button.click()
                    time.sleep(random.choice(sequence))
                except:
                    pass

                break
            else:
                continue

        #click more
        try:
            more_button = driver.find_element_by_class_name('ulBlueLinks')
            more_button.click()
            time.sleep(random.choice(sequence))
        except:
            pass

        #check if this is the same as last time
        if last_comment == comments(driver):
            print('Last Comment is Current Comment')
            break
        else:
            try:
                attraction_features['comments']+=(comments(driver))
            except:
                attraction_features['comments']+=''

            last_comment = comments(driver)

            time.sleep(random.choice(sequence))






In [ ]:
df= pd.DataFrame(attractions)

In [ ]:
df['comments'][0]

problem is if it doesn't have comments, it shouldnt go through the loop and waste time.

How to only get english? Should we get that?

That try statement is going to be to long. 

if there's a next on q and a and none on comments

In [ ]:
"""
1- If count > 3: press next
2- if there is a next button
3- Click more 
4- 


"""

In [ ]:
x = 3
for i in range(10):
    if x == 7:
        print(0)
        break
    else:
        print(x)
        x+=1
        

In [ ]:
count = 0
for i in range(3)

In [ ]:
"""
The problem is that we need a definitive reason to press next.
We want at most 3 comments.
If page is the same as last time. Break. 




"""

## Testing

In [ ]:
athens_links[0]

In [ ]:
driver.get(athens_links[6])

In [ ]:
# attraction_features['comments'] = None

In [ ]:
for i in driver.find_elements_by_class_name('next'):
                    print(i.text)
                    if i.text=='Next':
                        next_button = i
                        next_button.click()

In [ ]:
attraction_features = {}

In [ ]:
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

last_comment = comments(driver)

In [ ]:
sequence = [x/10 for x in range(10, 20)]

In [ ]:
for i in tqdm(range(3)):    
    #click next
    for j in tqdm(driver.find_elements_by_class_name('next')):
        if j.text == 'Next':
            next_button = j
            try:
                next_button.click()
                time.sleep(random.choice(sequence))
            except:
                pass
            
            break
        else:
            continue
    
    #click more
    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))
    except:
        pass

    #check if this is the same as last time
    if last_comment == comments(driver):
        print('Last Comment is Current Comment')
        break
    else:
        try:
            attraction_features['comments']+=(comments(driver))
        except:
            attraction_features['comments']+=''

        last_comment = comments(driver)
        
        time.sleep(random.choice(sequence))
        
        
        
        
        

In [ ]:
len(attraction_features['comments'])

In [ ]:
attraction_features = {}

In [ ]:
    try:
        #loop through 1st 3 pages if they have a next
        for next_comments in range(4):
            if driver.find_element_by_class_name('next')== None:
                break
            else:
                for i in driver.find_elements_by_class_name('next'):
                    print(i.text)
                    if i.text=='Next':
                        next_button = i
                        next_button.click()
                        

            time.sleep(random.choice(sequence))

            try:
                more_button = driver.find_element_by_class_name('ulBlueLinks')
                more_button.click()
                time.sleep(random.choice(sequence))

            except:
                pass


            #get comments
            # add to dictionary

            driver.implicitly_wait(5)
            try:
                attraction_features['comments']+=(comments(driver))
            except:
                attraction_features['comments']+=''

            time.sleep(random.choice(sequence))
        
    except:
        pass

In [ ]:
attraction_features = {}
# add to dictionary
try:
    attraction_features['name'] = find_name(driver)
except:
    attraction_features['name'] = None

# add to dictionary
try:
    attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
except:
    attraction_features['no_things_to_do'] = None

# add to dictionary
try:
    attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
except:
    attraction_features['sights_landmarks'] = None

# add to dictionary
try:
    attraction_features['address'] = find_address(driver)
except:
    attraction_features['address'] = None

# add to dictionary
try:
    attraction_features['summary_overview'] = find_summary_overview(driver)
except:
    attraction_features['summary_overview'] = None

# add to dictionary
try:
    attraction_features['num_reviews'] = number_of_reviews(driver)
except:
    attraction_features['num_reviews'] = None

# add to dictionary
try:
    attraction_features['about'] = about_box(driver)
except:
    attraction_features['about'] = None

# add to dictionary
try:
    attraction_features['popular_mentions'] = popular_mentions(driver)
except:
    attraction_features['popular_mentions'] = None

    
try:
    more_button = driver.find_element_by_class_name('ulBlueLinks')
    more_button.click()
    
except:
    pass
    
# add comments to dictionary
driver.implicitly_wait(5)
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

    
    
try:
    for i in driver.find_elements_by_class_name('next'):
        if i.text=='Next':
            next_button = i
            next_button.click()
            break
    
#add to list of dictionaries
attractions.append(attraction_features)

sequence = [x/10 for x in range(10, 15)]
time.sleep(random.choice(sequence))

In [ ]:
# first click more
# then get comments!


sequence = [x/10 for x in range(15, 30)]

#loop through 1st 3 pages if they have a next
for next_comments in range(4):
    if driver.find_element_by_class_name('next')== None:
        break
    else:
        for i in driver.find_elements_by_class_name('next'):
            if i.text=='Next':
                next_button = i
                next_button.click()
    
    time.sleep(random.choice(sequence))
    
    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))

    except:
        pass
        
    
    #get comments
    # add to dictionary
    
    driver.implicitly_wait(5)
    try:
        attraction_features['comments']+=(comments(driver))
    except:
        attraction_features['comments']+=''
        
    time.sleep(random.choice(sequence))

# Santorini Activities

## Santorini Links

In [276]:
from tqdm import tnrange, tqdm_notebook as tqdm

In [284]:
attraction_links = []
miss_links = []

In [304]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette

In [286]:
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [287]:
for i in tqdm(range(4)):
#if there is a next button, continue to click and scrape
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        print()
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
              
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    sequence = [x/10 for x in range(5, 10)]
    time.sleep(random.choice(sequence))

##

for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)
        
##

fixed_links = []

##

for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

##

df = pd.DataFrame(fixed_links)

##

df.to_pickle('santorini_links.pkl')

##

In [298]:
santorini_links = list(set(fixed_links))

In [283]:
attraction_links = []
miss_links = []

##

driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

##

# Don't run attraction_links in same location
# attraction_links = []

#319 things to do in Athens
#Range= number of pages to go to
for i in tqdm(range(4)):
#if there is a next button, continue to click and scrape
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        print()
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
              
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    sequence = [x/10 for x in range(5, 10)]
    time.sleep(random.choice(sequence))

##

for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)
        
##

fixed_links = []

##

for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

##

df = pd.DataFrame(fixed_links)

##

df.to_pickle('santorini_links.pkl')

##

## Santorini Activities

In [405]:
attractions = []

In [299]:
len(santorini_links)

120

In [410]:
test = santorini_links[5:]

In [411]:
print(len(santorini_links), len(test))

120 115


In [412]:
# Scraper
#definitions will change
# log in first 
# then do a test run to get correct definitions

for i in tqdm(test):
    driver.get(i)
    time.sleep(random.choice(sequence))
    time.sleep(random.choice(sequence))

    
    attraction_features = {}
    
    attraction_features['link'] = i
    
    try:
        morebutton(driver)
        time.sleep(random.choice(sequence)/2)
    except:
        pass
    
    time.sleep(random.choice(sequence))
    
    
    
#     def find_meta_data(driver):
#     return driver.find_element_by_class_name('masthead_h1').text
    
    # add to dictionary
    try:
        attraction_features['meta'] = find_meta_data(driver)
    except:
        attraction_features['meta'] = None

    
    # add to dictionary
    try:
        attraction_features['name'] = find_name(driver)
    except:
        attraction_features['name'] = None

    # add to dictionary
    try:
        attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
    except:
        attraction_features['no_things_to_do'] = None

    # add to dictionary
    try:
        attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
    except:
        attraction_features['sights_landmarks'] = None

    # add to dictionary
    try:
        attraction_features['address'] = find_address(driver)
    except:
        attraction_features['address'] = None

    # add to dictionary
    try:
        attraction_features['summary_overview'] = find_summary_overview(driver)
    except:
        attraction_features['summary_overview'] = None

    # add to dictionary
    try:
        attraction_features['num_reviews'] = number_of_reviews(driver)
    except:
        attraction_features['num_reviews'] = None

    # add to dictionary
    try:
        attraction_features['about'] = about_box(driver)
    except:
        attraction_features['about'] = None

    # add to dictionary
    try:
        attraction_features['popular_mentions'] = popular_mentions(driver)
    except:
        attraction_features['popular_mentions'] = None
    
    
    # add comments to dictionary
    
    try:
        attraction_features['comments'] = comments(driver)
    except:
        attraction_features['comments'] = None
    
    
    # last_comment == comments(driver)
    try:
        first_comment = comments(driver)
    except:
        pass
    
    

    
    #load more comments if there is any
    for i in (range(3)):

        try:
            if nextbutton(driver) == False:
                break
            else:
                nextbutton(driver)
                time.sleep(random.choice(sequence))

                try:
                    morebutton(driver)
                    time.sleep(random.choice(sequence)/3)
                except:
                    pass

                time.sleep(random.choice(sequence))

                try:
                    if first_comment == comments(driver):
                        break
                    else:
                        first_comment = comments(driver)
                        
                except:
                    pass
                
                try:
                    attraction_features['comments']+=(comments(driver))
                except:
                    attraction_features['comments']+=''    
                    
                
        except:
            break
        
        
        try:
            #checks to make sure we're not recopying comments
            if last_comment == comments(driver):
                break
        except:
            pass
    #add to list of dictionaries
    attractions.append(attraction_features)



In [414]:
df = pd.DataFrame(attractions)

In [ ]:
df.to_pickle('santorini_')